# playground for gfn sprint

In [29]:
import gc
import hydra
import json
import sys
import os
import importlib
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from peft import (
    get_peft_model,
    prepare_model_for_kbit_training,
)

In [3]:
# make proof_flow package available for imports
sys.path.append(os.path.abspath(os.path.join('..')))

In [55]:
from proof_flow.src.utils import (
    repo_root,
    prepare_environment_for_lean_dojo,
    get_config,
    set_up_padding,
)

In [5]:
prepare_environment_for_lean_dojo()

In [12]:
get_config.cache_clear() # clear cache to avoid stale config
main_cfg = get_config()

## Multiple Adapters
testing having new peft adapters, then loading another set of adapters

In [23]:
# main_cfg.task.model.name
# for testing purposes I need to have a RM adapter ready
main_cfg.task.model.name = "EleutherAI/llemma_7b"

In [56]:
tokenizer = AutoTokenizer.from_pretrained(main_cfg.task.model.name)
model = AutoModelForCausalLM.from_pretrained(
    main_cfg.task.model.name,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]
Some parameters are on the meta device device because they were offloaded to the cpu.


In [57]:
set_up_padding(model, tokenizer)

Tokenizer pad token ID: 32016
Model pad token ID: 32016


In [58]:
rm_cfg = get_config(config_name="verifier_training")

In [59]:
peft_config = hydra.utils.instantiate(
    rm_cfg.sft.model.lora
)
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=256, target_modules='all-linear', lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [60]:
model = get_peft_model(
    model,
    peft_config,
    adapter_name="gfn_policy",
)

In [61]:
model.active_adapters

['gfn_policy']

In [62]:
rm_adapter_hf_id = "msho/llemma_sft"

In [63]:
type(model)

peft.peft_model.PeftModelForCausalLM

In [64]:
model.load_adapter(
    rm_adapter_hf_id,
    adapter_name="reward",
)

/opt/conda/envs/gfn/lib/python3.10/site-packages/torch/nn/modules/module.py:2068: UserWarning: for base_model.model.model.layers.3.self_attn.q_proj.lora_A.reward.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/opt/conda/envs/gfn/lib/python3.10/site-packages/torch/nn/modules/module.py:2068: UserWarning: for base_model.model.model.layers.3.self_attn.q_proj.lora_B.reward.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying fr

_IncompatibleKeys(missing_keys=['base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.gfn_policy.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.gfn_policy.weight', 'base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'base

In [21]:
del model
del tokenizer
torch.cuda.empty_cache()
gc.collect()

489